In [2]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#import test_func as tf
%matplotlib inline

In [3]:
files = glob.glob("train/*.csv")
data_list = []
for file in files:
    data_list.append(pd.read_csv(file, index_col=0))
df = pd.concat(data_list)

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [7]:
#df = tf.data_pre(df)
#model = tf.model_lgb(df)

In [9]:
def data_pre(df):
    nonnull_list = []
    for col in df.columns:
        nonnull = df[col].count()
        if nonnull == 0:
            nonnull_list.append(col)
    df = df.drop(nonnull_list, axis=1)

    df = df.drop("市区町村名", axis=1)

    df = df.drop("種類", axis=1)

    dis = {
        "30分?60分":45,
        "1H?1H30":75,
        "2H?":120,
        "1H30?2H":105
    }
    df["最寄駅：距離（分）"] = df["最寄駅：距離（分）"].replace(dis).astype(float)

    df["面積（㎡）"] = df["面積（㎡）"].replace("2000㎡以上", 2000).astype(float)


    y_list = {}
    for i in df["建築年"].value_counts().keys():
        if "平成" in i:
            num = float(i.split("平成")[1].split("年")[0])
            year = 33 - num
        if "令和" in i:
            num = float(i.split("令和")[1].split("年")[0])
            year = 3 - num
        if "昭和" in i:
            num = float(i.split("昭和")[1].split("年")[0])
            year = 96 - num
        y_list[i] = year
    y_list["戦前"] = 76
    df["建築年"] = df["建築年"].replace(y_list)

    year = {
        "年第1四半期": ".25",
        "年第2四半期": ".50",
        "年第3四半期": ".75",
        "年第4四半期": ".99"
    }
    year_list = {}
    for i in df["取引時点"].value_counts().keys():
        for k, j in year.items():
            if k in i:
                year_rep = i.replace(k, j)
        year_list[i] = year_rep
    df["取引時点"] = df["取引時点"].replace(year_list).astype(float)
    
    for col in ["都道府県名", "地区名", "最寄駅：名称", "間取り", "建物の構造", "用途", "今後の利用目的", "都市計画", "改装", "取引の事情等"]:
                df[col] = df[col].astype("category")
    
    return df

In [10]:
df = data_pre(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 707485 entries, 40103204 to 12010481
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   市区町村コード       707485 non-null  int64   
 1   都道府県名         707485 non-null  category
 2   地区名           706818 non-null  category
 3   最寄駅：名称        704776 non-null  category
 4   最寄駅：距離（分）     684350 non-null  float64 
 5   間取り           682328 non-null  category
 6   面積（㎡）         707485 non-null  float64 
 7   建築年           686959 non-null  float64 
 8   建物の構造         689270 non-null  category
 9   用途            640702 non-null  category
 10  今後の利用目的       342163 non-null  category
 11  都市計画          687864 non-null  category
 12  建ぺい率（％）       683596 non-null  float64 
 13  容積率（％）        683596 non-null  float64 
 14  取引時点          707485 non-null  float64 
 15  改装            641456 non-null  category
 16  取引の事情等        18824 non-null   category
 17  取引価格（総額）_log  707485

In [11]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as mae

df_train, df_val =train_test_split(df, test_size=0.2)

col = "取引価格（総額）_log"
train_y = df_train[col]
train_x = df_train.drop(col, axis=1)

val_y = df_val[col]
val_x = df_val.drop(col, axis=1)

trains = lgb.Dataset(train_x, train_y)
valids = lgb.Dataset(val_x, val_y)

params = {
    "objective": "regression",
    "metrics": "mae"
}

model = lgb.train(params, trains, valid_sets=valids, num_boost_round=1000, early_stopping_rounds=100)


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.022835 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 11660
[LightGBM] [Info] Number of data points in the train set: 565988, number of used features: 17


/opt/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/opt/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Info] Start training from score 7.226258
[1]	valid_0's l1: 0.253538
Training until validation scores don't improve for 100 rounds
[2]	valid_0's l1: 0.23713
[3]	valid_0's l1: 0.222721
[4]	valid_0's l1: 0.210063
[5]	valid_0's l1: 0.19872
[6]	valid_0's l1: 0.188729
[7]	valid_0's l1: 0.179983
[8]	valid_0's l1: 0.172143
[9]	valid_0's l1: 0.165066
[10]	valid_0's l1: 0.158923
[11]	valid_0's l1: 0.153355
[12]	valid_0's l1: 0.148293
[13]	valid_0's l1: 0.143965
[14]	valid_0's l1: 0.140046
[15]	valid_0's l1: 0.13654
[16]	valid_0's l1: 0.133574
[17]	valid_0's l1: 0.130621
[18]	valid_0's l1: 0.128027
[19]	valid_0's l1: 0.125373
[20]	valid_0's l1: 0.123179
[21]	valid_0's l1: 0.120909
[22]	valid_0's l1: 0.118997
[23]	valid_0's l1: 0.117079
[24]	valid_0's l1: 0.115523
[25]	valid_0's l1: 0.114022
[26]	valid_0's l1: 0.112325
[27]	valid_0's l1: 0.111006
[28]	valid_0's l1: 0.109871
[29]	valid_0's l1: 0.108773
[30]	valid_0's l1: 0.107638
[31]	valid_0's l1: 0.106135
[32]	valid_0's l1: 0.105336
[

[278]	valid_0's l1: 0.0781603
[279]	valid_0's l1: 0.0781522
[280]	valid_0's l1: 0.0781424
[281]	valid_0's l1: 0.0781344
[282]	valid_0's l1: 0.0781313
[283]	valid_0's l1: 0.0781266
[284]	valid_0's l1: 0.0781147
[285]	valid_0's l1: 0.0781082
[286]	valid_0's l1: 0.0781059
[287]	valid_0's l1: 0.0781006
[288]	valid_0's l1: 0.0780951
[289]	valid_0's l1: 0.0780865
[290]	valid_0's l1: 0.0780787
[291]	valid_0's l1: 0.0780765
[292]	valid_0's l1: 0.0780743
[293]	valid_0's l1: 0.078055
[294]	valid_0's l1: 0.0780388
[295]	valid_0's l1: 0.0780373
[296]	valid_0's l1: 0.0780348
[297]	valid_0's l1: 0.0780167
[298]	valid_0's l1: 0.0780082
[299]	valid_0's l1: 0.0780048
[300]	valid_0's l1: 0.0779828
[301]	valid_0's l1: 0.077948
[302]	valid_0's l1: 0.0779386
[303]	valid_0's l1: 0.0779292
[304]	valid_0's l1: 0.0779237
[305]	valid_0's l1: 0.0779269
[306]	valid_0's l1: 0.0779267
[307]	valid_0's l1: 0.0779206
[308]	valid_0's l1: 0.0779177
[309]	valid_0's l1: 0.0779013
[310]	valid_0's l1: 0.0778959
[311]	valid_

[558]	valid_0's l1: 0.0767847
[559]	valid_0's l1: 0.076784
[560]	valid_0's l1: 0.0767803
[561]	valid_0's l1: 0.0767812
[562]	valid_0's l1: 0.0767813
[563]	valid_0's l1: 0.0767766
[564]	valid_0's l1: 0.0767686
[565]	valid_0's l1: 0.0767644
[566]	valid_0's l1: 0.076763
[567]	valid_0's l1: 0.0767633
[568]	valid_0's l1: 0.0767572
[569]	valid_0's l1: 0.0767569
[570]	valid_0's l1: 0.0767515
[571]	valid_0's l1: 0.0767514
[572]	valid_0's l1: 0.0767473
[573]	valid_0's l1: 0.076744
[574]	valid_0's l1: 0.0767459
[575]	valid_0's l1: 0.0767424
[576]	valid_0's l1: 0.0767435
[577]	valid_0's l1: 0.0767359
[578]	valid_0's l1: 0.0767364
[579]	valid_0's l1: 0.0767307
[580]	valid_0's l1: 0.0767315
[581]	valid_0's l1: 0.0767283
[582]	valid_0's l1: 0.0767269
[583]	valid_0's l1: 0.0767262
[584]	valid_0's l1: 0.0767209
[585]	valid_0's l1: 0.0767218
[586]	valid_0's l1: 0.076721
[587]	valid_0's l1: 0.0767206
[588]	valid_0's l1: 0.0767228
[589]	valid_0's l1: 0.0767183
[590]	valid_0's l1: 0.0767188
[591]	valid_0'

[835]	valid_0's l1: 0.0763889
[836]	valid_0's l1: 0.076387
[837]	valid_0's l1: 0.0763871
[838]	valid_0's l1: 0.0763874
[839]	valid_0's l1: 0.0763881
[840]	valid_0's l1: 0.0763849
[841]	valid_0's l1: 0.0763844
[842]	valid_0's l1: 0.0763852
[843]	valid_0's l1: 0.0763803
[844]	valid_0's l1: 0.0763802
[845]	valid_0's l1: 0.0763799
[846]	valid_0's l1: 0.0763726
[847]	valid_0's l1: 0.0763722
[848]	valid_0's l1: 0.0763733
[849]	valid_0's l1: 0.0763733
[850]	valid_0's l1: 0.0763742
[851]	valid_0's l1: 0.0763739
[852]	valid_0's l1: 0.0763728
[853]	valid_0's l1: 0.0763731
[854]	valid_0's l1: 0.0763691
[855]	valid_0's l1: 0.0763657
[856]	valid_0's l1: 0.0763645
[857]	valid_0's l1: 0.0763659
[858]	valid_0's l1: 0.0763644
[859]	valid_0's l1: 0.0763637
[860]	valid_0's l1: 0.0763635
[861]	valid_0's l1: 0.0763597
[862]	valid_0's l1: 0.0763603
[863]	valid_0's l1: 0.0763591
[864]	valid_0's l1: 0.0763595
[865]	valid_0's l1: 0.0763557
[866]	valid_0's l1: 0.0763554
[867]	valid_0's l1: 0.076356
[868]	valid_

In [12]:
vals = model.predict(val_x)
mae(vals, val_y)

0.07626118197466981

In [13]:
df_test = pd.read_csv("test.csv", index_col=0)
df_test = data_pre(df_test)

In [14]:
predict = model.predict(df_test)
df_test["取引価格（総額）_log"] = predict
df_test[["取引価格（総額）_log"]].to_csv("submit_test.csv")

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 707485 entries, 40103204 to 12010481
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   市区町村コード       707485 non-null  int64   
 1   都道府県名         707485 non-null  category
 2   地区名           706818 non-null  category
 3   最寄駅：名称        704776 non-null  category
 4   最寄駅：距離（分）     684350 non-null  float64 
 5   間取り           682328 non-null  category
 6   面積（㎡）         707485 non-null  float64 
 7   建築年           686959 non-null  float64 
 8   建物の構造         689270 non-null  category
 9   用途            640702 non-null  category
 10  今後の利用目的       342163 non-null  category
 11  都市計画          687864 non-null  category
 12  建ぺい率（％）       683596 non-null  float64 
 13  容積率（％）        683596 non-null  float64 
 14  取引時点          707485 non-null  float64 
 15  改装            641456 non-null  category
 16  取引の事情等        18824 non-null   category
 17  取引価格（総額）_log  707485

In [16]:
pd.DataFrame(model.feature_importance(), index=val_x.columns, columns=["importance"]).sort_values("importance", ascending=False)

,importance
地区名,8713
最寄駅：名称,6532
建築年,3295
取引時点,3024
面積（㎡）,1944
最寄駅：距離（分）,1852
間取り,854
市区町村コード,723
都道府県名,569
改装,491
